https://habr.com/ru/articles/482050/ идею взял из этой статьи. Описана идея слабо, часть пришлось придумывать самому, вроде что-то интересное получилось

Список содержимого:
1. Функции-хелперы, можно не смотреть
2. Загрузка модели
3. Функция-обрезчик (портянка кода, но по-другому никак)
4. Тесты (валидация-дообучение-валидация) + в самом конце написал выжимку

In [ ]:
from google.colab import drive


drive.mount('/content/drive')

Mounted at /content/drive


# Всякое

In [ ]:
!pip install -q torchmetrics torchinfo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 7.5 MB/s eta 0:00:00


In [ ]:
import torch
from torch import nn
from itertools import groupby


class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, pool_ksize=(2, 2)):
        super(ConvBlock, self).__init__()

        self.block = nn.Sequential(nn.Conv2d(in_channels, out_channels, 3, padding='same'),
                                   nn.LeakyReLU(0.1),
                                   nn.BatchNorm2d(out_channels),
                                   nn.MaxPool2d(pool_ksize))

    def forward(self, x):
        return self.block(x)


class CRNN(nn.Module):
    def __init__(self, alphabet_len):
        super(CRNN, self).__init__()

        self.feature_extractor = nn.Sequential(ConvBlock(1, 32),
                                               ConvBlock(32, 64, (2, 1)),
                                               ConvBlock(64, 64),
                                               ConvBlock(64, 128),
                                               ConvBlock(128, 256, (2, 1)))
        self.lstm1 = nn.LSTM(258, 256, batch_first=True)
        self.lstm2 = nn.LSTM(256, 256, batch_first=True)

        self.fc = nn.Sequential(nn.Linear(256, alphabet_len+1),
                                nn.Softmax(dim=2))

    def forward(self, x1, x2):
        f1 = self.feature_extractor(x1).squeeze()
        f1 = torch.permute(f1, (0, 2, 1))

        x = torch.cat([f1, x2], dim=2)

        x, _ = self.lstm1(x)
        assert x.isnan().sum() == 0, 'nans detected'
        x, _ = self.lstm2(x)
        assert x.isnan().sum() == 0, 'nans detected'
        x = self.fc(x)
        assert x.isnan().sum() == 0, 'nans detected'

        return x


def decode_texts(logits, alphabet):
    """Decodes CRNN output with given alphabet and whitelist

    Args:
        logits: np.ndarray, CRNN output
        alphabet: str, alphabet CRNN was trained on
    Returns:
        list of predictions
    """
    best_path_indices = np.argmax(logits, axis=-1)
    best_chars_collapsed = [[alphabet[k-1] for k, _ in groupby(e) if k != 0] for e in best_path_indices]
    return [''.join(e) for e in best_chars_collapsed]

In [ ]:
def ctc_loss_log_differentiable_torch(log_logits: torch.FloatTensor, targets: torch.LongTensor,
                                      input_lengths: torch.Tensor, target_lengths: torch.Tensor, device,
                                      blank_idx=0, dtype_to_use=torch.float32) -> torch.float32:
    """
    log_logits: np.ndarray of shape (B, T, C)
    targets: np.ndarray of shape (B, L,)
    """

    B, T = log_logits.shape[0], log_logits.shape[1]
    S = 2 * targets.shape[1] + 1

    zero = torch.finfo(dtype_to_use).min

    # insert blanks between every pair of labels and add them to start and end of the seq
    extended_targets = torch.stack([torch.full_like(targets, blank_idx), targets], dim=-1).flatten(start_dim=-2)
    extended_targets = torch.cat([extended_targets, torch.full((B, 1), blank_idx, device=device)], dim=-1)
    # due to the paper formula for alpha_t(s) we must know where labels repeat and where the blanks are
    # in the extended label seq
    targets_difference_mask = torch.cat([torch.full((B, 2), False, device=device), extended_targets[:, 2:] != extended_targets[:, :-2]], dim=-1)

    # initialize alphas array to keep track of previous alphas
    # (also add 2 to the second dim so our s-2 and s-1 vectorized calculations won't get IndexError)
    log_alphas = torch.full((B, T, S+2), zero, dtype=dtype_to_use, device=device)

    # every accountable prefix starts either with a blank or the first symbol of the target,
    # so we initialize alphas in the following way (remember about S+2)
    log_alphas[:, 0, 2] = log_logits[:, 0, blank_idx]
    log_alphas[:, 0, 3] = log_logits[torch.arange(B), 0, targets[:, 0]]

    for t in range(1, T):
        # remember we're in log space so log(a*b) = log(a) + log(b)
        # here formula must be mathematically reworked.

        log_alphas[:, t, 2:] = (torch.gather(log_logits[:, t], -1, extended_targets) +
                                torch.logsumexp(torch.stack([log_alphas[:, t-1, 2:], log_alphas[:, t-1, 1: -1],
                                                             torch.where(targets_difference_mask,
                                                                         log_alphas[:, t-1, :-2], zero)]), dim=0))

    temp = torch.gather(log_alphas[np.arange(B), input_lengths-1], -1,
                        torch.stack([2 + target_lengths * 2 - 1, 2 + target_lengths * 2], dim=-1))

    return -torch.mean(torch.logsumexp(temp, dim=-1))

In [ ]:
def train_model(model, epochs, train_loader, lr, device='cpu'):
  model.to(device)
  model.train()

  optimizer = torch.optim.NAdam(model.parameters(), lr=lr)
  metric = CharErrorRate()

  criterion = ctc_loss_log_differentiable_torch

  for epoch in range(epochs):
    train_loss = 0
    val_loss = 0

    train_cer = 0
    val_cer = 0

    for i, ((x1, x2), y) in enumerate(train_loader):
      x1 = x1.to(device)
      x2 = x2.to(device)
      y = y.to(device)

      optimizer.zero_grad()
      y_pred = model(x1, x2)

      input_lengths = torch.full((y_pred.shape[0],), y_pred.shape[1]).to(device)
      target_lengths = torch.sum(y != 0, axis=1)

      loss = criterion(torch.log(y_pred), y, input_lengths, target_lengths, device=device)
      loss.backward()

      optimizer.step()

      train_loss += loss.item()
      train_cer += metric(decode_texts(y_pred.detach().cpu().numpy(), alphabet),
                          [''.join(alphabet[k-1] for k, _ in groupby(e) if k != 0) for e in y.cpu().numpy().astype(int)]).item()

      print(f'\rEpoch {epoch}, {i+1}/{len(train_loader)}, loss: {round(train_loss/(i+1), 6)}, cer: {round(train_cer/(i+1), 6)}', end='')
    print()

  return model

In [ ]:
from torchmetrics.text import CharErrorRate
from itertools import groupby
from tqdm import tqdm
import time

def validate_model(model, dataloader, device='cpu'):
  model.eval()

  criterion = ctc_loss_log_differentiable_torch
  metric = CharErrorRate()
  loss = 0
  cer_value = 0
  cumtime = 0

  with torch.no_grad():
    for i, ((x1, x2), y) in tqdm(enumerate(dataloader)):
      x1 = x1.to(device)
      x2 = x2.to(device)
      y = y.to(device)

      start = time.time()
      y_pred = model(x1, x2)
      cumtime += time.time() - start

      input_lengths = torch.full((y_pred.shape[0],), y_pred.shape[1]).to(device)
      target_lengths = torch.sum(y != 0, axis=1)
      loss += criterion(torch.log(y_pred), y, input_lengths, target_lengths, device=device).item()
      cer_value += metric(decode_texts(y_pred.detach().cpu().numpy(), alphabet),
                        [''.join(alphabet[k-1] for k, _ in groupby(e) if k != 0) for e in y.cpu().numpy().astype(int)]).item()

  print()

  return cumtime / len(dataloader), loss / len(dataloader), cer_value / len(dataloader)

In [ ]:
from torch.utils.data import Dataset, DataLoader


class OCRDataset(Dataset):
    def __init__(self, images, abits, labels):
        super(OCRDataset, self).__init__()

        self.images = images
        self.abits = abits
        self.labels = labels

    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self, idx):
        return (torch.FloatTensor(self.images[idx]).unsqueeze(0), torch.FloatTensor(self.abits[idx])), torch.IntTensor(self.labels[idx])

In [ ]:
import h5py
import pandas as pd
import numpy as np

with h5py.File('/content/drive/MyDrive/CRNN for long fields/common_fields_images.h5') as f:
    images = f['images'][:]
    additional_bits = f['additional_bit'][:]

with open('/content/drive/MyDrive/CRNN for long fields/common_fields_labels.txt', encoding='cp1251') as f:
    markup = [e.strip() for e in f.readlines()]


def encode_texts(texts):
    def _label_to_num(label, alphabet):
        label_num = []
        for ch in label:
            label_num.append(alphabet.find(ch) + 1)
        return np.array(label_num)

    # alphabet = ''.join(sorted(pd.Series(texts).apply(list).apply(pd.Series).stack().unique()))
    alphabet = ''.join(sorted(set(''.join(texts))))

    nums = np.zeros([len(texts), max([len(text) for text in texts])], dtype='int64')
    for i, text in enumerate(texts):
        nums[i][:len(text)] = _label_to_num(text, alphabet)

    return nums, alphabet

labels_encoded, alphabet = encode_texts(markup)
images = images.astype('float64') / 255

additional_bits_expanded = np.zeros((len(images), 50, 2))
additional_bits_expanded[:, :, additional_bits] = 1

np.random.seed(42)

train_indices = np.random.choice(np.arange(images.shape[0]), int(images.shape[0]*0.8), replace=False)
val_indices = [e for e in np.arange(images.shape[0]) if e not in train_indices]

assert len(set(train_indices) & set(val_indices)) == 0
assert len(set(train_indices) | set(val_indices)) == images.shape[0]

train_imgs = images[train_indices]
val_imgs = images[val_indices]

train_abits = additional_bits_expanded[train_indices]
val_abits = additional_bits_expanded[val_indices]

train_labels = labels_encoded[train_indices]
val_labels = labels_encoded[val_indices]

train_dataset = OCRDataset(train_imgs, train_abits, train_labels)
val_dataset = OCRDataset(val_imgs, val_abits, val_labels)

train_loader = DataLoader(train_dataset, batch_size=128)
val_loader = DataLoader(val_dataset, batch_size=128)

# Загрузка и обрезание модели

In [ ]:
import torch
from torchinfo import summary

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_ = CRNN(len(alphabet))
model_.load_state_dict(torch.load('/content/drive/MyDrive/Методы компрессии/crnn_common_fields_.pt', map_location=torch.device(device)))
summary(model_, input_size=[(32, 1, 32, 400), (32, 50, 2)], device=device)

Layer (type:depth-idx)                   Output Shape              Param #
CRNN                                     [32, 50, 46]              --
├─Sequential: 1-1                        [32, 256, 1, 50]          --
│    └─ConvBlock: 2-1                    [32, 32, 16, 200]         --
│    │    └─Sequential: 3-1              [32, 32, 16, 200]         384
│    └─ConvBlock: 2-2                    [32, 64, 8, 200]          --
│    │    └─Sequential: 3-2              [32, 64, 8, 200]          18,624
│    └─ConvBlock: 2-3                    [32, 64, 4, 100]          --
│    │    └─Sequential: 3-3              [32, 64, 4, 100]          37,056
│    └─ConvBlock: 2-4                    [32, 128, 2, 50]          --
│    │    └─Sequential: 3-4              [32, 128, 2, 50]          74,112
│    └─ConvBlock: 2-5                    [32, 256, 1, 50]          --
│    │    └─Sequential: 3-5              [32, 256, 1, 50]          295,680
├─LSTM: 1-2                              [32, 50, 256]             

In [ ]:
print(dict(zip(['batch_time', 'loss', 'metric'], [round(e, 6) for e in validate_model(model_, val_loader, device=device)])))

26it [00:01, 14.22it/s]


{'batch_time': 0.032099, 'loss': 0.623181, 'metric': 0.049073}


Пока что пришлось отключить прунинг рекуррентных и полносвязных слоев потому что там где-то есть проблемка (скорее всего в прунинге рекуррентных слоев)

In [ ]:
from ast import Module
from copy import deepcopy

model = deepcopy(model_)

layers_to_prune = list()

def get_layers(module):
    children = list(module.children())
    return [module] if len(children) == 0 else [ci for c in children for ci in get_layers(c)]


model.to('cpu')

layers = get_layers(model)
layers_to_prune = list(filter(lambda x: isinstance(x, (nn.Conv2d, nn.BatchNorm2d, nn.LSTM, nn.Linear)), layers))[:-1]
# layers_to_prune = list(filter(lambda x: isinstance(x, (nn.Conv2d, nn.BatchNorm2d)), layers))
layers_to_prune

[Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same),
 BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same),
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same),
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=same),
 BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=same),
 BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 LSTM(258, 256, batch_first=True)]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fraction = 0.1

sorted_indices = None
prev_layer_output_size = 0
prunable_counter = 0

for i, layer in enumerate(layers_to_prune):
  # first process indices from previous layer
  if sorted_indices is not None:
    if isinstance(layer, nn.Conv2d):
      layer.in_channels = len(sorted_indices)
      layer.weight = torch.nn.Parameter(layer.weight[:, sorted_indices])
    elif isinstance(layer, nn.BatchNorm2d):
      layer.num_features = len(sorted_indices)

      layer.weight = torch.nn.Parameter(layer.weight[sorted_indices])
      layer.bias = torch.nn.Parameter(layer.bias[sorted_indices])
      layer.running_mean = layer.running_mean[sorted_indices]
      layer.running_var = layer.running_var[sorted_indices]
    elif isinstance(layer, nn.LSTM):
      # its here due to conv extracted features concatenation with second input
      sorted_indices = list(set(np.arange(layer.input_size)) - (set(np.arange(prev_layer_output_size)) - set(sorted_indices.numpy())))

      for attr in dir(layer):
        if attr.startswith('weight_ih'):
          layer.input_size = len(sorted_indices)
          setattr(layer, attr, torch.nn.Parameter(getattr(layer, attr)[:, sorted_indices]))

    elif isinstance(layer, nn.Linear):
      layer.in_features = len(sorted_indices)
      layer.weight = torch.nn.Parameter(layer.weight[:, sorted_indices])

  if len(layers_to_prune) - i <= 1:
    break

  # third prune
  if isinstance(layer, nn.Conv2d):
    l1_kernelwise = torch.sum(torch.abs(layer.weight), dim=(1, 2, 3)).detach()
    sorted_indices = torch.argsort(l1_kernelwise)[int(fraction*layer.out_channels):]
    sorted_indices = torch.sort(sorted_indices).values

    prev_layer_output_size = layer.out_channels
    layer.out_channels = len(sorted_indices)
    layer.weight = torch.nn.Parameter(layer.weight[sorted_indices])
    layer.bias = torch.nn.Parameter(layer.bias[sorted_indices])

  elif isinstance(layer, nn.LSTM):
    prev_layer_output_size = layer.hidden_size

    for attr in dir(layer):
      if attr.startswith('weight_ih'):
        weight = getattr(layer, attr).view(layer.hidden_size, 4, -1)
        l1 = torch.sum(torch.abs(weight), dim=(1, 2))
        sorted_indices = torch.argsort(l1)[int(fraction*layer.hidden_size):]
        sorted_indices = torch.sort(sorted_indices).values

        bias_name = attr.replace('weight', 'bias')
        setattr(layer, attr, torch.nn.Parameter(weight[sorted_indices].view(-1, weight.shape[-1])))
        setattr(layer, bias_name, torch.nn.Parameter(getattr(layer, bias_name)[sorted_indices]))

    for attr in dir(layer):
      if attr.startswith('weight_hh'):
        weight = getattr(layer, attr)[:, sorted_indices].view(layer.hidden_size, 4, -1)
        l1 = torch.sum(torch.abs(weight), dim=(1, 2))
        sorted_indices = torch.argsort(l1)[int(fraction*layer.hidden_size):]
        sorted_indices = torch.sort(sorted_indices).values

        bias_name = attr.replace('weight', 'bias')
        setattr(layer, attr, torch.nn.Parameter(weight[sorted_indices].view(-1, weight.shape[-1])))
        setattr(layer, bias_name, torch.nn.Parameter(getattr(layer, bias_name)[sorted_indices]))

    layer.hidden_size = len(sorted_indices)

  elif isinstance(layer, nn.Linear):
    l1 = torch.sum(torch.abs(layer.weight), dim=1)
    sorted_indices = torch.argsort(l1)[int(fraction*layer.in_features):]
    sorted_indices = torch.sort(sorted_indices).values

    layer.weight = torch.nn.Parameter(layer.weight[sorted_indices])

    layer.out_features = len(sorted_indices)

# Тесты

Внутрь модельки засунул assert, который проверяет, что в выходе нет nan'ов. Почему-то они там возникают, если прунить каналы в рекуррентной части сети, это я позже попробую исправить

In [ ]:
summary(model, input_size=[(32, 1, 32, 400), (32, 50, 2)], device=device)

Layer (type:depth-idx)                   Output Shape              Param #
CRNN                                     [32, 50, 46]              --
├─Sequential: 1-1                        [32, 231, 1, 50]          --
│    └─ConvBlock: 2-1                    [32, 29, 16, 200]         --
│    │    └─Sequential: 3-1              [32, 29, 16, 200]         348
│    └─ConvBlock: 2-2                    [32, 58, 8, 200]          --
│    │    └─Sequential: 3-2              [32, 58, 8, 200]          15,312
│    └─ConvBlock: 2-3                    [32, 58, 4, 100]          --
│    │    └─Sequential: 3-3              [32, 58, 4, 100]          30,450
│    └─ConvBlock: 2-4                    [32, 116, 2, 50]          --
│    │    └─Sequential: 3-4              [32, 116, 2, 50]          60,900
│    └─ConvBlock: 2-5                    [32, 231, 1, 50]          --
│    │    └─Sequential: 3-5              [32, 231, 1, 50]          241,857
├─LSTM: 1-2                              [32, 50, 256]             

In [ ]:
print(dict(zip(['batch_time', 'loss', 'metric'], [round(e, 6) for e in validate_model(model, val_loader, device=device)])))

26it [00:01, 15.09it/s]


{'batch_time': 0.029588, 'loss': 0.904908, 'metric': 0.055323}


In [ ]:
torch.autograd.set_detect_anomaly(True)

model = train_model(model, 2, train_loader, lr=1e-4, device=device)

Epoch 0, 103/103, loss: 0.214567, cer: 0.038388
Epoch 1, 103/103, loss: 0.152685, cer: 0.037062


In [ ]:
print(dict(zip(['batch_time', 'loss', 'metric'], [round(e, 6) for e in validate_model(model, val_loader, device=device)])))

26it [00:01, 15.24it/s]


{'batch_time': 0.029938, 'loss': 0.549252, 'metric': 0.047336}


* Было: {'batch_time': 0.0312, 'loss': 0.623181, 'metric': 0.049073}
* После прунинга (без дообучения) стало: {'batch_time': 0.029588, 'loss': 0.904908, 'metric': 0.055323}
* После прунинга (с дообучением) стало: {'batch_time': 0.025779, 'loss': 0.551292, 'metric': 0.047284}

https://stackoverflow.com/questions/52945427/pytorch-manually-setting-weight-parameters-with-numpy-array-for-gru-lstm